# Feasible Solutions for Problem 5 of Midterm Exam

Since there was a lot of out-of-the-box, creative algorithms written by students in the midterm exam, this notebook implements a set of diverse algorithms to solve the same problem, to make the job of assestment and grading a bit easier.

## The Problem

Consider string S. Suggest a dynamic programming algorithm that finds the longest non-overlapping, repeating substring in $O(n^2)$.

Example:
S = AACAACAACA
Output: AACA

### Base Class for Algorithms

In [14]:
from abc import ABC, abstractmethod
from typing import List, Tuple

class LongestRepeatingSubstring(ABC):
    def __init__(self):
      pass

    @abstractmethod
    def find_longest_repeating_substring(self, string) -> List[str]:
        pass


## Solving The Problem Using Vanilla Dynamic Programming

This method constructs a DP table with lengths of longest common suffixes for substrings ending at each pair of indices. It ensures non-overlapping by checking if the characters at the current indices in the string are equal and only then, does it increment the length of the common suffix from the previous indices. The longest repeating substring is then retrieved by maintaining a running maximum length of the common suffix and the end index of the maximum length substring.

#### Recursive Formula

The recursive formula for solution using Vanilla Dynamic Programming is:

$$
\text{dp}[i][j] =
\begin{cases}
\text{dp}[i-1][j-1] + 1 & \text{if: } s[i-1] = s[j-1] \\
0 & \text{otherwise}
\end{cases}
$$

#### Complexity

The time complexity is $O(n^2)$ due to the double iteration over the string length, and the space complexity is $O(n^2)$ for storing the DP table. The DP table is used to keep track of the lengths of the longest common suffixes of the substrings ending at each pair of indices.


In [15]:
class LongestRepeatingSubstringVanillaDP(LongestRepeatingSubstring):
    def find_longest_repeating_substring(self, string):
        n = len(string)
        dp = [[0 for _ in range(n+1)] for _ in range(n+1)]
        max_length = 0
        end_index = 0

        for i in range(1, n+1):
            for j in range(i+1, n+1):
                if string[i-1] == string[j-1]:
                    dp[i][j] = dp[i-1][j-1] + 1
                    if dp[i][j] > max_length:
                        max_length = dp[i][j]
                        end_index = i
                else:
                    dp[i][j] = 0

        return [string[end_index - max_length : end_index]] if max_length > 0 else []


In [16]:
def run_test_cases(lrs: LongestRepeatingSubstring, test_cases: List[Tuple[str, List[str]]]):
    for i, (test_case, expected_output) in enumerate(test_cases):
        print(f"Test case {i+1}:")
        print(f"Input: {test_case}")
        output = lrs.find_longest_repeating_substring(test_case)
        print(f"Output: {output}")
        assert set(output) == set(expected_output), f"Expected {expected_output}, but got {output}"
        print("Output matches expected output.")
        print()


test_cases = [
    ("AG", []),  # Length 2, no repeating substring
    ("ATAT", ["AT"]),  # Length 4, repeating substring "AT"
    ("TCGTCG", ["TCG"]),  # Length 6, repeating substring "TCG"
    ("CGCGAATTCGCG", ["CGCG"]),  # Length 12, repeating substring "CGCG"
    ("TACGTACGTTGCAACGTACGT", ["ACGTACGT"])  # Length 20, repeating substring "ACGTACGT"
]



In [17]:
lrs = LongestRepeatingSubstringVanillaDP()

run_test_cases(lrs, test_cases)

Test case 1:
Input: AG
Output: []
Output matches expected output.

Test case 2:
Input: ATAT
Output: ['AT']
Output matches expected output.

Test case 3:
Input: TCGTCG
Output: ['TCG']
Output matches expected output.

Test case 4:
Input: CGCGAATTCGCG
Output: ['CGCG']
Output matches expected output.

Test case 5:
Input: TACGTACGTTGCAACGTACGT
Output: ['ACGTACGT']
Output matches expected output.



## Suffix Tree/Array with Dynamic Programming

This method constructs a suffix array and an LCP array for the string. It then finds the longest repeating substring by finding the maximum value in the LCP array. The time complexity is $O(n^2 \log n)$ due to the sorting of the suffixes, and the space complexity is $O(n)$ for storing the suffix array and the LCP array.
The recursive formula for the longest common prefix (LCP) is:

#### Recursive Formula

$$
\text{lcp}[i] =
\begin{cases}
0 & \text{if: } i = 0 \\
\text{lcp}[i-1] + 1 & \text{if } s1[i] = s2[i] \\
0 & \text{otherwise}
\end{cases}
$$



In [18]:
class LrsSuffixTree(LongestRepeatingSubstring):
    def find_longest_repeating_substring(self, string: str) -> List[str]:
        n = len(string)
        # Construct a suffix array
        suffixes = sorted([(string[i:], i) for i in range(n)])
        suffix_array = [i for _, i in suffixes]

        # Construct a longest common prefix (LCP) array
        lcp = [0] * n
        for i in range(1, n):
            lcp[i] = self._lcp(suffixes[i-1][0], suffixes[i][0])

        # Find the longest repeating substring
        max_len = max(lcp)
        idx = lcp.index(max_len)
        start = suffix_array[idx]
        end = start + max_len
        return [string[start:end]] if max_len > 0 else []

    def _lcp(self, s1: str, s2: str) -> int:
        i = 0
        while i < len(s1) and i < len(s2) and s1[i] == s2[i]:
            i += 1
        return i


In [19]:
lrs = LrsSuffixTree()

run_test_cases(lrs, test_cases)

Test case 1:
Input: AG
Output: []
Output matches expected output.

Test case 2:
Input: ATAT
Output: ['AT']
Output matches expected output.

Test case 3:
Input: TCGTCG
Output: ['TCG']
Output matches expected output.

Test case 4:
Input: CGCGAATTCGCG
Output: ['CGCG']
Output matches expected output.

Test case 5:
Input: TACGTACGTTGCAACGTACGT
Output: ['ACGTACGT']
Output matches expected output.



## Solving The Problem Using Local Sequence Alignment

This method aligns the string with itself and fills a DP table with lengths of longest common suffixes for substrings ending at each pair of indices. It ensures non-overlapping by checking that the current substring isn't within the previous substring's range. The longest repeating substring is then retrieved by finding the maximum value in the DP table.

#### Recursive Formula

The recursive formula for solution using local sequence alignment is:

$$
\text{dp}[i][j] =
\begin{cases}
\text{dp}[i-1][j-1] + 1 & \text{if: } s[i-1] = s[j-1] \text{ and } (j-i) > \text{dp}[i-1][j-1] \\
0 & \text{otherwise}
\end{cases}
$$

#### Complexity

The time complexity is $O(n^2)$ due to the double iteration over the string length, and the space complexity is $O(n^2)$ for storing the DP table.

In [20]:
class LrsLocalSeqAlignment(LongestRepeatingSubstring):
    def find_longest_repeating_substring(self, string: str) -> List[str]:
        n = len(string)
        dp = [[0 for _ in range(n+1)] for _ in range(n+1)]
        max_length = 0
        end_index = 0

        for i in range(1, n+1):
            for j in range(i+1, n+1):
                if string[i-1] == string[j-1] and (j-i) > dp[i-1][j-1]:
                    dp[i][j] = dp[i-1][j-1] + 1
                    if dp[i][j] > max_length:
                        max_length = dp[i][j]
                        end_index = i
                else:
                    dp[i][j] = 0

        if max_length > 0:
            return [string[end_index - max_length : end_index]]
        else:
            return []


In [21]:
lrs = LrsLocalSeqAlignment()

run_test_cases(lrs, test_cases)

Test case 1:
Input: AG
Output: []
Output matches expected output.

Test case 2:
Input: ATAT
Output: ['AT']
Output matches expected output.

Test case 3:
Input: TCGTCG
Output: ['TCG']
Output matches expected output.

Test case 4:
Input: CGCGAATTCGCG
Output: ['CGCG']
Output matches expected output.

Test case 5:
Input: TACGTACGTTGCAACGTACGT
Output: ['ACGTACGT']
Output matches expected output.



## Solving The Problem Using Trie and Dynamic Programming

This method constructs a Trie with all suffixes of the input string. It then aligns the string with itself and fills a DP table with lengths of longest common prefixes for substrings ending at each pair of indices. It ensures non-overlapping by checking that the current substring isn't within the previous substring's range. The longest repeating substring is then retrieved by finding the maximum value in the DP table and the corresponding substring in the Trie.

#### Recursive Formula

The recursive formula for solution using Trie and Dynamic Programming is:

$$
\text{dp}[i][j] =
\begin{cases}
\text{dp}[i-1][j-1] + 1 & \text{if: } s[i-1] = s[j-1] \text{ and } \text{dp}[i-1][j-1] < (j-i) \\
0 & \text{otherwise}
\end{cases}
$$

#### Complexity

The time complexity is $O(n^2)$ due to the double iteration over the string length, and the space complexity is $O(n)$ for storing the Trie and the result set. The DP table is used to keep track of the lengths of the longest common prefixes of the substrings ending at each pair of indices, which requires $O(n^2)$ space.


In [22]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.indexes = []

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, string: str, index: int):
        node = self.root
        for char in string:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
            node.indexes.append(index)

    def search(self, string: str):
        node = self.root
        for char in string:
            if char not in node.children:
                return []
            node = node.children[char]
        return node.indexes

In [23]:
class LongestRepeatingSubstringDPAndTrie(LongestRepeatingSubstring):
    def find_longest_repeating_substring(self, string: str) -> List[str]:
        n = len(string)
        trie = Trie()
        dp = [[0 for j in range(n+1)] for i in range(n+1)]
        res_length = 0
        res_set = set()

        for i in range(n):
            indexes = trie.search(string[i:])
            trie.insert(string[i:], i)
            for j in indexes:
                if i > 0 and j > 0:
                    dp[i][j] = dp[i-1][j-1] + 1
                else:
                    dp[i][j] = 1
                if dp[i][j] > res_length:
                    res_length = dp[i][j]
                    res_set.clear()
                    res_set.add(string[i-res_length+1:i+1])
                elif dp[i][j] == res_length:
                    res_set.add(string[i-res_length+1:i+1])

        return list(res_set) if res_set else []


In [24]:
lrs = LongestRepeatingSubstringDPAndTrie()

run_test_cases(lrs, test_cases)

Test case 1:
Input: AG
Output: []
Output matches expected output.

Test case 2:
Input: ATAT
Output: ['AT']
Output matches expected output.

Test case 3:
Input: TCGTCG
Output: ['TCG']
Output matches expected output.

Test case 4:
Input: CGCGAATTCGCG
Output: ['CGCG']
Output matches expected output.

Test case 5:
Input: TACGTACGTTGCAACGTACGT
Output: ['ACGTACGT']
Output matches expected output.



## Non Dynamic Programming Solution: Using Hash Table

#### The Idea:

1. Initialize a hash table to store the hashes of the substrings.
2. Start with a window size of 1 and slide it over the entire string to hash all substrings of size 1.
3. If a hash already exists in the hash table, check if the substring is non-overlapping with the existing substring. If it is, update the longest repeating non-overlapping substring.
4. Increment the window size and repeat steps 2-3 until the window size is equal to the length of the string.

#### Complexity

The time complexity is $O(n^2)$ because we are sliding a window over the entire string for each window size from 1 to n. The space complexity is also $O(n^2)$ because in the worst case, we might store all substrings of the string in the hash table.

In [25]:
class LrsWithRollingHash:
    def find_longest_repeating_substring(self, string):
        n = len(string)
        max_length = 0
        max_substring = ""

        for length in range(1, n):
            hash_table = {}
            for i in range(n - length + 1):
                substr = string[i:i+length]
                if substr in hash_table:
                    if i - hash_table[substr] >= length:
                        max_length = length
                        max_substring = substr
                else:
                    hash_table[substr] = i

        return [max_substring] if max_length > 0 else []


In [26]:
lrs = LrsWithRollingHash()

run_test_cases(lrs, test_cases)

Test case 1:
Input: AG
Output: []
Output matches expected output.

Test case 2:
Input: ATAT
Output: ['AT']
Output matches expected output.

Test case 3:
Input: TCGTCG
Output: ['TCG']
Output matches expected output.

Test case 4:
Input: CGCGAATTCGCG
Output: ['CGCG']
Output matches expected output.

Test case 5:
Input: TACGTACGTTGCAACGTACGT
Output: ['ACGTACGT']
Output matches expected output.

